### FINAL PROJECT

Hello, for this module (week 4), I would like to make a recommendation system, where you should put what you rate more high about living in your current neighborhood, and with that I could tell you witch neighborhood it's your better option, for the sake of simplicity we are going to think that you only care about the nearest business.
In order to achieve this we can think use ideas:
1.- People who lives in (random neighborhood), wich neighborhood we can recommend to him.
2.- People who goes to vacation to a new country and want to feel a little bit more in home.
I think both of them are a nice business problem to solve with Python, because in my work we travel a lot inside of the country (Mexico), and we have the chance to choose the hotel, but some times it's hard choose a good place.
In the other hand I have never been in other country but If I find my self in that situation I would like to have a  similar tool

In [14]:
import numpy as np
import pandas as pd 
import requests
#!pip3 install geocoder
import geocoder
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

ModuleNotFoundError: No module named 'geopy'

#### We will use pandas method "set_option" when we need to analyze dataframes whit a lot of columns or rows

In [15]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### The most simple way for scraping web it's the method read_html, but we can use library's like beautifulsoup/requests

In [16]:
import pandas as pd
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
torontoDF=pd.read_html(url, header=0)[0]
print(torontoDF.shape)
torontoDF.head()

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

#### As you can see the dataframe have many "Not assigned" values at the Borough and Neighbourhood columns, we should delete these rows, because they don't give us usefull information

In [17]:
torontoDF = torontoDF[~torontoDF['Borough'].isin(['Not assigned'])]
print(torontoDF.shape)
torontoDF.reset_index(inplace=True)
torontoDF.drop('index', axis=1, inplace=True)
torontoDF.head()

NameError: name 'torontoDF' is not defined

#### If we find 'Not assigned' values in the Neighbourhood column we should replace them with the Borough value, the best way to do this could be the for statment because you can create conditions and check row by row

In [ ]:
for index, row in torontoDF.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']


print(torontoDF.shape)
torontoDF.head(15)

#### The last step is the more complicated, first of all we group the information by the Postcode and then use the aggregate method for the neighbourhood's, with this we can make the neighbourhood's join between the duplicated Postcodes

In [ ]:
torontoDF = torontoDF.groupby('Postcode').agg({'Borough':'first',
                               'Neighbourhood': ', '.join}).reset_index()

print(torontoDF.shape)
torontoDF.head()

In [ ]:
torontoDF.shape

In [ ]:
toronto = pd.read_csv('./Documents/Geospatial_Coordinates.csv')

In [ ]:
toronto.shape

In [ ]:
toronto.head()

In [ ]:
neighborhoods = toronto.merge(torontoDF, left_on='Postal Code', right_on='Postcode')

#### Now we need the latitud and longitude, with geocoder library and a while loop we can do it, but it's reallyyyy slow.

In [ ]:
neighborhoods.head()

In [ ]:
# neighborhoods = torontoDF.copy()

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()),neighborhoods.shape[0]))

#### Now we need the latitud and longitude from Toronto, because the map needs inicial coordinates

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

#### With the latitud and longitude we center the map and with the latitud's and longitude's from our data frame we can plot all the Neighbourhood's

In [ ]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Define Foursquare Credentials

### Explore Neighborhoods in Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighbourhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

#### Now with the foursquare API we can get the nearby venues for each neighborhood.

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

#### Count venues for each Neighbourhood

In [ ]:
toronto_venues.groupby('Neighbourhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

### Analyze Each Neighborhood's

In [2]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

NameError: name 'toronto_venues' is not defined

In [ ]:
toronto_onehot.shape

### Now we summarize all the categories into their Neighbourhood's

In [3]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

NameError: name 'toronto_onehot' is not defined

In [4]:
toronto_grouped.shape

NameError: name 'toronto_grouped' is not defined

#### Principals feautures for each Neighbourhood

In [5]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

NameError: name 'toronto_grouped' is not defined

In [6]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [7]:
toronto_grouped.shape

NameError: name 'toronto_grouped' is not defined

#### The most common feature for each Neighbourhood

In [8]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']
neighborhoods_venues_sorted
for ind in np.arange(100):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

NameError: name 'toronto_grouped' is not defined